# Automating data wrangling

## Introduction

Sometimes we require a "one off" solution to a unique data analysis problem. In this situation, we write code to do a particular analysis on a particular data set. Then, if the analysis is part of a publication, we make the code and data publically available and... we're done.

Often, however, we require a ***reusable*** solution that operates on data of a given format even though some of the particulars, such as sample size or variable names, might change. In this case, we want our code to be "dynamic" in the sense that it should be able to handle any anticipated changes to the details of the input data.

Here, we'll tackle the same problem as last time – reformatting a data set from a cumbersome format into a more useful and "tidy" format.

Learning goals:

* write reusable code for a data wrangling problem
* create a function to make the code handy to use

## Import pandas and look at the data from last time

In [1]:
import pandas as pd

Read in the data from last time.

In [2]:
my_input_data = pd.read_csv('datasets/017DataFile.csv')

Take a peek to remind ourselves of the structure.

In [3]:
my_input_data.head()

,Male Mutant,Female Mutant,Male Wild Type,Female Wild Type
0,10.485451,8.250013,20.127063,25.946384
1,11.747948,8.453839,20.068147,23.464870
2,13.412580,9.706605,21.215148,22.989480
3,12.910095,9.522116,20.706416,25.324376
4,10.367770,8.583212,18.074795,22.607487


In this data set, there are two "independent variables", sex and genotype of laboratory rats, and one "dependent variable", response time. The data are formatted such that each column contains the data from a unique combination of the two independent variables, *i.e.* a "cell" of the experimental design.

This format might seem to make sense, but it's actually not very flexible. For analysis purposes, it's generally better to have data in a format that obeys a couple of rules:

* each row should correspond to a single observation (measurement)
* each column should correspond to a single variable

Data in this format are also referred to as "tidy".

So in this case, our goal is to take the above data and put it into a format like this:

| response time | sex | genotype |
| ---| --- | --- |
| rt value | male or female | wild or not |

Once the data are in this format, we can easily use our tools to do things like compare wild to mutant, or compare wild to mutant only in females, etc.

Last time, we stacked the reaction time values into a single column using pandas functions. This relied on us knowing and "hard coding" the column names ("Male Mutant", etc.). If we're going to automate things, we want our code to be agnostic about these. One way would be to somehow read the column names into variables and work with them somehow... 

But since we already know how to "play legos" with numpy arrays, which are just numbers so we don't need to worry about column names, let's try that!

In [4]:
import numpy as np

Pandas dataframes know how to convert themselves to numpy arrays. They have a `to_numpy()` method. This is going to pull just the numbers out of our dataframe, ignoring the column labels and row indexs.

In [5]:
raw_data = my_input_data.to_numpy() 

Let's take a look!

In [6]:
raw_data

array([[10.48545088,  8.2500131 , 20.12706278, 25.94638414],
       [11.74794775,  8.45383932, 20.06814699, 23.46487013],
       [13.41258004,  9.70660484, 21.21514789, 22.98948034],
       [12.91009526,  9.52211638, 20.70641578, 25.32437595],
       [10.36777045,  8.58321246, 18.07479515, 22.60748688],
       [11.69842177,  9.83500171, 20.36762403, 23.05218737],
       [11.58315277, 10.53209602, 20.15252058, 25.3690367 ],
       [11.44734892,  9.39416641, 19.39247581, 23.37270897],
       [10.85227619,  8.73947266, 18.52434071, 25.21564644],
       [11.28589742, 10.89239399, 20.32502629, 24.99050453]])

## Get some useful infomation from the original data

So far so good! Now we are going to put the data into the format we want. To automate this, we are going to get the number of observations in each group – the number of rows – and the number of groups – the number of columns – and store them in variables.

In [7]:
obs_per_grp, grps = raw_data.shape
print("We have ", obs_per_grp, " observations per group and ", grps, " groups.")

We have  10  observations per group and  4  groups.


Now we'll calculate the total number of observations, which is also how long we want our new data frame to be.

In [8]:
new_length = obs_per_grp*grps
print("We have ", new_length, " total observations.")

We have  40  total observations.


## Build our response time (dependent variable) column

We could now play legos "by hand", stacking the columns of our numpy array on top of each other to make a new array (and we already know how to do that). 

But we can also take advantage of the fact that one of the things numpy arrays know how to do – one of the methods they have – is change their shape. So we'll take our `obs` by `cols` array and `numpy.reshape()` into a `new_length` by 1 array.

What this command does is read out the data values from the original array one-by-one, and places them in a new array of a shape you specify. The only catch is that the total number of cells in the new array has to be the same as in the old array – in other words, each and every data value has to have one and only one place to go in the new array. Which makes sense.

In [9]:
values_col = np.reshape(raw_data, (new_length, 1))

I called it `values_col` because it will eventually become the values column of our new pandas data frame.

Let's see if that worked:

In [10]:
values_col

array([[10.48545088],
       [ 8.2500131 ],
       [20.12706278],
       [25.94638414],
       [11.74794775],
       [ 8.45383932],
       [20.06814699],
       [23.46487013],
       [13.41258004],
       [ 9.70660484],
       [21.21514789],
       [22.98948034],
       [12.91009526],
       [ 9.52211638],
       [20.70641578],
       [25.32437595],
       [10.36777045],
       [ 8.58321246],
       [18.07479515],
       [22.60748688],
       [11.69842177],
       [ 9.83500171],
       [20.36762403],
       [23.05218737],
       [11.58315277],
       [10.53209602],
       [20.15252058],
       [25.3690367 ],
       [11.44734892],
       [ 9.39416641],
       [19.39247581],
       [23.37270897],
       [10.85227619],
       [ 8.73947266],
       [18.52434071],
       [25.21564644],
       [11.28589742],
       [10.89239399],
       [20.32502629],
       [24.99050453]])

Nice! But let's make sure that worked. What we want is for the columns of the original data to be stacked on top of one another. Is that what we have?

Nope, it's not right. What happened is that the values got read out *left to right, top to bottom* (or row-wise) and placed into the new array one-by-one. But what we want is for the values to be read *top to bottom, left to right* (or columnwise). We can make this happen with the `order=` argument of `numpy.reshape()`.

In [11]:
values_col = np.reshape(raw_data, (new_length, 1), order = 'F')

Let's make sure that worked:

In [12]:
values_col

array([[10.48545088],
       [11.74794775],
       [13.41258004],
       [12.91009526],
       [10.36777045],
       [11.69842177],
       [11.58315277],
       [11.44734892],
       [10.85227619],
       [11.28589742],
       [ 8.2500131 ],
       [ 8.45383932],
       [ 9.70660484],
       [ 9.52211638],
       [ 8.58321246],
       [ 9.83500171],
       [10.53209602],
       [ 9.39416641],
       [ 8.73947266],
       [10.89239399],
       [20.12706278],
       [20.06814699],
       [21.21514789],
       [20.70641578],
       [18.07479515],
       [20.36762403],
       [20.15252058],
       [19.39247581],
       [18.52434071],
       [20.32502629],
       [25.94638414],
       [23.46487013],
       [22.98948034],
       [25.32437595],
       [22.60748688],
       [23.05218737],
       [25.3690367 ],
       [23.37270897],
       [25.21564644],
       [24.99050453]])

**Yay!** It did!

**Useless trivia**: Two of Ye Olde Major Programming Languages are **C** (used mainly by programmers) and **Fortran** (used mainly by scientists). C (the language used to write Python) uses row-wise indexing, whereas Fortran uses columnwise indexing. That's why "F" is used to specify columnwise indexing above: the "F" is for "Fortran".

Minor annoying thing: (there is always at least one that pops up in any coding task, amirite?) `values_col` is a (40x1) 2-dimensional numpy array but, when we go to build our new data frame, we'll need it to be a 40 long (40,) 1-dimensional array. 

This actually comes up so often that `numpy` has a `squeeze()` function to squeeze the dimension of length one into nothingness. It turns (n, 1) things into (n,) things.

Let's check the shape of our new array:

In [13]:
values_col.shape

(40, 1)

Now let's squeeze the (uneeded and unwanted) column dimension into oblivion:

In [14]:
values_col = np.squeeze(values_col)

And check the shape again:

In [15]:
values_col.shape

(40,)

Okay, that worked, now onto...

## Building the independent variable columns

What we require is that the levels our two independent variables repeat themselves in the right order down their respective columns. We could certainly type this in by hand, but that would be really annoying to change if we required new labels later on or something. 

We could also use `for()` loops; they are designed for exactly such repetitive tasks after all. That might look something like this:

In [16]:
gen_var = list()                     # create a python list 
for i in range(new_length) :         # loop through all observations
    if i < new_length/2 :            # for the first half, ...
        gen_var.append("wildtype")   # set to male
    else :                           # otherwise...
        gen_var.append("mutant")     # set to female

In [17]:
print(gen_var)

['wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'wildtype', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant', 'mutant']


We'd have to get a little bit more fancy with our `if...` to create the sex variable, that'd be the idea.

But pandas provides easy ways to repeat and stack things (numpy does too), so let's try those. The two will use are

* `pandas.Series.repeat()` 
* `pandas.concat()`

Note: When you see `pandas.Series.somefunction()` or `pandas.DataFrame.somefunction()` in the documentation, that means that all Series or DataFrames know how to do `somefunction()`. So if you had a Series named `Phred`, you would say `Phred.somefunction()` to use `somefunction()`.

### Make the genetic strain variable

In the way we have formatted the data, genetic strain is the "outer" variable, in that it only changes once as we go down the data set: all the wildtypes are on top, and all mutants are on the bottom. The sex variable is the "inner" variable, because it changes once within each value of strain, so it needs to three times as we go down the data set.

This is arbitrary and has nothing to do with the experimental design; we could have formatted the data such that the roles were reversed.

What we will do is 

* make a short series containing the two levels of our variable
* repeat each value to make the long series 
* deal with annoying index values (there's always something...)

In [23]:
strain = pd.Series(['wildtype', 'mutant'])  # make the short series
strain = strain.repeat(2*obs_per_grp)       # repeat each over two cell's worth of data
strain = strain.reset_index(drop=True)      # reset the series's index value

Let's see if that worked:

In [27]:
print(strain)

0     wildtype
1     wildtype
2     wildtype
3     wildtype
4     wildtype
5     wildtype
6     wildtype
7     wildtype
8     wildtype
9     wildtype
10    wildtype
11    wildtype
12    wildtype
13    wildtype
14    wildtype
15    wildtype
16    wildtype
17    wildtype
18    wildtype
19    wildtype
20      mutant
21      mutant
22      mutant
23      mutant
24      mutant
25      mutant
26      mutant
27      mutant
28      mutant
29      mutant
30      mutant
31      mutant
32      mutant
33      mutant
34      mutant
35      mutant
36      mutant
37      mutant
38      mutant
39      mutant
dtype: object


### Make the sex variable

As the sex variable is the inner variable, we need it have ['male'..., 'female'...] within each outer block of genotype. So what we'll do is make one block of ['male'..., 'female'...] and then just stack two copies of that to make our variable. So the steps are

* make a short series containing the two levels of our variable (just like above)
* repeat it (just like above)
* stack two copies on top of each other (dropping the annoying indexes in the process)

In [28]:
sexes = pd.Series(['male', 'female'])             # make the short series
sexes = sexes.repeat(obs_per_grp)                 # repeat each over one cell's worth of data
sexes = pd.concat([sexes]*2, ignore_index=True)   # stack or "concatonate" two copies

### Build our new data frame!

Data frames are created in pandas by handing it data it can make sense of. There are various ways to accomplish this, and one handy one is to hand it data in a "column label 1 : data 1, column label 2 : data 2, ..." format. 

We can accomplish this with a python "dictionary", which is a thing associates a label (the "word") with a value or set of values or whatever (the "definition"). They are very useful, so let's take a look at a simple example before we use one to build out data frame. You create a dictionary using curly braces, and then use colons to bind each word or `key` with its definition or `value`. Commas separate each key-value pair.

In [33]:
myData = {"name": "Larry", "rank": "full", "years": 30, "bikes": 5, "motorcycles": 2, "teslas": 1}

In [34]:
myData["name"]

'Larry'

In [35]:
myData["bikes"]

5

So a dictionary associates a label with data values. **Perfect!**

Time to build our data frame!

In [36]:
my_tidy_data = pd.DataFrame(      # invoke creation
    {                             # start the dictionary with a {
        "RTs": values_col,        # assign each variable to a label
        "sex": sexes,
        "strain": strain
    }                             # end the dictionary with a }
)                                 # end of creation

Note that the formatting above is just to make the columns we're creating more obvious and human-readable. This will work too:

In [38]:
my_tidy_data = pd.DataFrame({"RTs": values_col, "sex": sexes, "strain": strain})

It's just not as pretty.

Let's look at our creation!

In [39]:
my_tidy_data

,RTs,sex,strain
0,10.485451,male,wildtype
1,11.747948,male,wildtype
2,13.412580,male,wildtype
3,12.910095,male,wildtype
4,10.367770,male,wildtype
5,11.698422,male,wildtype
6,11.583153,male,wildtype
7,11.447349,male,wildtype
8,10.852276,male,wildtype
9,11.285897,male,wildtype


Yay! We win!

**Important point:** Crucially, *the above code doesn't rely on us knowing much about the input data ahead of time*. As long as it's a pandas data frame that contains numerical values, the code will run. It's automatic.

## Look at new data with more observations with same code

We'll make this code self-contained, so it can be run without running anything above. We'll also add comments, so that future-us can read the code more easily without having to wade through the notebook text above.

In [ ]:
# import our libraries
import pandas as pd
import numpy as np

my_input_data = pd.read_csv('datasets/018DataFile.csv')  # read the data

raw_data = my_input_data.to_numpy()                      # convert to numpy array

obs, grps = raw_data.shape                               # get the number of rows and columns

Check the size of the new data real quick:

In [ ]:
print("We have ", obs, " observations per group and ", grps, " groups.")

In [ ]:
new_length = obs*grps                                    # compute total number of observations

values_col = np.reshape(raw_data, (new_length, 1), 
                        order = 'F')                     # reshape the array
values_col = np.squeeze(values_col)                      # squeeze to make 1D

# construct the inner grouping variable
sexes = pd.Series(['male', 'female'])                    # define the levels
sexes = sexes.repeat(obs)                                # make one cycle of the levels
sexes = pd.concat([sexes]*2, ignore_index=True)          # and repeat the cycle, ditching the indexes

# construct the outer grouping variable
strain = pd.Series(['wildtype', 'mutant'])               # define the levels
strain = strain.repeat(2*obs)                            # make the one cycle
strain = strain.reset_index(drop=True)                   # drop the pesky index

# construct the data frame
my_new_tidy_data = pd.DataFrame(
    {
        "RTs": values_col,                               # make a column named RTs and put the values in
        "sex": sexes,                                    # ditto for sex
        "strain": strain                                 # and for genetic strain
    }    
)

In [ ]:
my_new_tidy_data

Success!

## Making the code even more functional

In [ ]:
def tidyMyData() :
    import pandas as pd
    import numpy as np

    my_input_data = pd.read_csv('datasets/018DataFile.csv')  # read the data

    raw_data = my_input_data.to_numpy()                      # convert to numpy array

    obs, grps = raw_data.shape                               # get the number of rows and columns

    new_length = obs*grps                                    # compute total number of observations

    values_col = np.reshape(raw_data, (new_length, 1), 
                            order = 'F')                     # reshape the array
    values_col = np.squeeze(values_col)                      # squeeze to make 1D

    # construct the inner grouping variable
    sexes = pd.Series(['male', 'female'])                    # define the levels
    sexes = sexes.repeat(obs)                                # make one cycle of the levels
    sexes = pd.concat([sexes]*2, ignore_index=True)     # and repeat the cycle, ditching the indexes

    # construct the outer grouping variable
    strain = pd.Series(['wildtype', 'mutant'])               # define the levels
    strain = strain.repeat(2*obs)                            # make the one cycle
    strain = strain.reset_index(drop=True)                   # drop the pesky index

    # construct the data frame
    my_new_tidy_data = pd.DataFrame(
        {
            "RTs": values_col,                               # make a column named RTs and put the values in
            "sex": sexes,                                    # ditto for sex
            "strain": strain                                 # and for genetic strain
        }    
    )
    
    return my_new_tidy_data

In [ ]:
datFromFun = tidyMyData()

In [ ]:
datFromFun